In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   #if like me you do not have a lot of memory in your GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "" #then these two lines force keras to use your CPU
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import h5py

import matplotlib as mpl
COLOR = 'white'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR
from tqdm.auto import tqdm

In [3]:
def read_data(path_dataset):
    data = {}
    time_init =     "Time:  0.00000E+00 y"
    time_final =    "Time:  5.00000E+00 y"
    number_datapoints = 0
    names_of_runs = []

    for file in tqdm(list(os.listdir(path_dataset))):
        try:
            path_data = os.path.join(path_dataset,file)
            if not os.path.isdir(path_data):
                continue
            print(file)
            names_of_runs.append(file)
            filename = os.path.join(path_data,"pflotran.h5") 
            with h5py.File(filename, "r") as f:
                for key, value in f[time_final].items():
                    if not key in data: # and not key=='Material_ID':
                        data[key] = []
                    #if not key=='Material_ID':
                    if key=='Liquid_Pressure [Pa]':
                        data[key].append(np.array(f[time_init]["Liquid_Pressure [Pa]"]))
                    else:
                        data[key].append(np.array(value))
            number_datapoints += 1

        except Exception as e:
            tqdm.write(f"lololololololoo: {e}")

    for key,value in data.items():
        data[key] = np.array(value)

    return data, number_datapoints, names_of_runs

In [4]:
def read_data_df(path_dataset):
    time_init =     "Time:  0.00000E+00 y"
    time_final =    "Time:  5.00000E+00 y"
    names_of_runs = []
    names_of_properties = []
    data = []

    for file in tqdm(list(os.listdir(path_dataset))):
        try:
            path_data = os.path.join(path_dataset,file)
            if not os.path.isdir(path_data):
                continue
            #print(file)
            names_of_runs.append(file)
            filename = os.path.join(path_data,"pflotran.h5") 
            interim_array = []
            with h5py.File(filename, "r") as f:
                for key, value in f[time_final].items():
                    if key=='Liquid_Pressure [Pa]':
                        interim_array.append(np.array(f[time_init]["Liquid_Pressure [Pa]"]))
                    else:
                        interim_array.append(np.array(value))
                names_of_properties = list(f[time_final].keys())
            data.append(interim_array)

        except Exception as e:
            tqdm.write(f"lololololololoo: {e}")
    
    df = pd.DataFrame(data=data, index=names_of_runs, columns=names_of_properties)

    return df

In [5]:
def aligned_colorbar(*args,**kwargs):
    cax = make_axes_locatable(plt.gca()).append_axes("right",size= 0.3,pad= 0.05)
    plt.colorbar(*args,cax=cax,**kwargs)
    
def plot_sample(data, sample_id, view="top"):
    n_dims = len(data)
    fig, axes = plt.subplots(n_dims,1,sharex=True,figsize=(20,3*n_dims))
    plt.figure(figsize= (20,3*n_dims))
    for i,(key,value) in zip(range(n_dims),data.items()):
        plt.sca(axes[i])
        field = value[sample_id]
        if len(field.shape) != 3:
            # no 3D data
            continue
        index = key.find(' [')
        title = key
        if index != -1:
            title = key[:index]
        plt.title(title)
        if view=="topish":
            plt.imshow(field[:,:,-3])
            plt.xlabel("y")
            plt.ylabel("x")
        elif view=="side":
            plt.imshow(field[11,:,::-1].T)
            plt.xlabel("y")
            plt.ylabel("z")
        elif view=="side_hp":
            plt.imshow(field[8,:,::-1].T)
            plt.xlabel("y")
            plt.ylabel("z")
        elif view=="top_hp":
            plt.imshow(field[:,:,8])
            plt.xlabel("y")
            plt.ylabel("x")
        elif view=="top":
            plt.imshow(field[:,:,-1])
            plt.xlabel("y")
            plt.ylabel("x")
        aligned_colorbar(label=key)
    plt.show()

def plot_sample_df(df, run_id, view="top"):
    n_dims = len(df.columns)
    fig, axes = plt.subplots(n_dims,1,sharex=True,figsize=(20,3*n_dims))
    plt.figure(figsize= (20,3*n_dims))
    for column, (i) in zip(df.columns, range(n_dims)):
        plt.sca(axes[i])
        field = df.at[run_id, column]
        if len(field.shape) != 3:
            # no 3D data
            continue
        index = column.find(' [')
        title = column
        if index != -1:
            title = column[:index]
        plt.title(title)
        if view=="topish":
            plt.imshow(field[:,:,-3])
            plt.xlabel("y")
            plt.ylabel("x")
        elif view=="side":
            plt.imshow(field[11,:,::-1].T)
            plt.xlabel("y")
            plt.ylabel("z")
        elif view=="side_hp":
            plt.imshow(field[8,:,::-1].T)
            plt.xlabel("y")
            plt.ylabel("z")
        elif view=="top_hp":
            plt.imshow(field[:,:,8])
            plt.xlabel("y")
            plt.ylabel("x")
        elif view=="top":
            plt.imshow(field[:,:,-1])
            plt.xlabel("y")
            plt.ylabel("x")
        aligned_colorbar(label=column)
    plt.show()

In [85]:
path_dir = "/home/pelzerja/Development/simulation_groundtruth_pflotran/Phd_simulation_groundtruth/approach2_dataset_generation_simplified"
dataset_name = "dataset_HDF5_test" #dataset_HDF5_uniformly_distributed_data
path_dataset = os.path.join(path_dir, dataset_name)

#data, number_datapoints, names_datapoints = read_data(path_dataset)
#print(data["Liquid_Pressure [Pa]"][0,:,:,:].shape)
#plot_sample(data,2,view="topish")
#print(number_datapoints)

df = read_data_df(path_dataset)
#plot_sample_df(df,"RUN_4", "top_hp")

# print(df)

100%|██████████| 7/7 [00:00<00:00, 169.43it/s]


**Preprocessing**

In [86]:
# find maximum alias hp
# print(np.where(data["Material_ID"]==np.max(data["Material_ID"])))

## data cleaning: cut of edges - to get rid of problems with boundary conditions
def data_cleaning(data):
    for key, (value) in data.items():
        data[key] = data[key][:,1:-1,1:-3,1:-1]
        print(f"{key} :{np.shape(data[key])}")
    return data

def data_cleaning_df(df):
    for label, content in df.items():
        for index in range(len(content)):
            content[index] = content[index][1:-1,1:-3,1:-1]
    return df

df = data_cleaning_df(df)
#plot_sample(df,"RUN_1",view="side_hp")

In [87]:
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

In [111]:
## shuffle data along "RUN_"-axis
# is already shuffled from reading-order but whatever
df_shuffled = df.sample(frac=1)

## split data into test and training data
train_size_percent = 0.8
train_size_abs = int(train_size_percent * df.shape[0]) # cuts instead of rounds

df_train = df_shuffled[:train_size_abs]
df_test = df_shuffled[train_size_abs:]

In [114]:
## data normalizing:
# prep data: normalize all input values (vel/pressure) and all output values (temp/vel?), each to its own min,max, std

# scaler-function for a row(=index) in the dataset
def get_max_min(array):
    return np.max(array), np.min(array)

def scale_data_row(data, column, desired_max = 1, desired_min = -1):
    max_list = []
    min_list = []
    for row in data.index:
        max_temp, min_temp = get_max_min(data.at[row, column])
        max_list.append(max_temp)
        min_list.append(min_temp)
    
    max_data = np.max(max_list)
    min_data = np.min(min_list)    
    for row in data.index:
        data_std = (data.at[row,column] - min_data) / (max_data - min_data)
        data.at[row, column] = data_std * (desired_max - desired_min) + desired_min
        print(row, column)
        #print(np.max(data.at[row, column]), np.min(data.at[row, column]))

    return data
#test = [[[1,2], [1,2]], [[3,3], [3,3]], [[1,2], [1,2]], [[-10,10], [4,4]]]
#test_res = scale_data_row(test)
#print(np.std(test_res))

def scale_data(data):
    for column in data.columns:
        data = scale_data_row(data, column)
    return data

df_train = scale_data(df_train)
df_test = scale_data(df_test)

# jut for manuel checking the values
#for column in df_train.columns:
#    for row in df_train.index:
#        print(column, row, np.min(df_train.at[row, column]), np.max(df_train.at[row, column]), np.std(df_train.at[row, column]))

1.0 -1.0
RUN_2 Liquid X-Velocity [m_per_y]
RUN_4 Liquid X-Velocity [m_per_y]
RUN_1 Liquid X-Velocity [m_per_y]
RUN_3 Liquid X-Velocity [m_per_y]
1.0 -1.0
RUN_2 Liquid Y-Velocity [m_per_y]
RUN_4 Liquid Y-Velocity [m_per_y]
RUN_1 Liquid Y-Velocity [m_per_y]
RUN_3 Liquid Y-Velocity [m_per_y]
1.0 -1.0
RUN_2 Liquid Z-Velocity [m_per_y]
RUN_4 Liquid Z-Velocity [m_per_y]
RUN_1 Liquid Z-Velocity [m_per_y]
RUN_3 Liquid Z-Velocity [m_per_y]
1.0 -1.0
RUN_2 Liquid_Pressure [Pa]
RUN_4 Liquid_Pressure [Pa]
RUN_1 Liquid_Pressure [Pa]
RUN_3 Liquid_Pressure [Pa]
1.0 -1.0
RUN_2 Material_ID
RUN_4 Material_ID
RUN_1 Material_ID
RUN_3 Material_ID
1.0 -1.0
RUN_2 Temperature [C]
RUN_4 Temperature [C]
RUN_1 Temperature [C]
RUN_3 Temperature [C]
6.65074867298968 -6.649386398890904
RUN_0 Liquid X-Velocity [m_per_y]
7.213143735226594 -4.985348762367756
RUN_0 Liquid Y-Velocity [m_per_y]
83.53336841228503 -7.012547789629725
RUN_0 Liquid Z-Velocity [m_per_y]
812254.1702669684 174851.14481186643
RUN_0 Liquid_Pressure

In [ ]:
## split data into input and labels
df_train_in = pd.DataFrame(df_train["Liquid_Pressure [Pa]"])
df_train_labels = pd.DataFrame(df_train[["Material_ID","Temperature [C]"]])
df_test_in = pd.DataFrame(df_test["Liquid_Pressure [Pa]"])
df_test_labels = pd.DataFrame(df_test[["Material_ID","Temperature [C]"]])

In [298]:
## parameters
#shuffle_buffer_size = 10
#
### seperate dataset into input data and corresponding output labels
#data_in = {}
#data_labels = {}
#
#data_in['Liquid_Pressure [Pa]'] = data['Liquid_Pressure [Pa]'] #maybe for help in beginning: 'Liquid X-Velocity [m_per_y]', 'Liquid Y-Velocity [m_per_y]', 'Liquid Z-Velocity [m_per_y]']
#data_in['Material_ID'] = data['Material_ID']
#data_labels['Temperature [C]'] = data['Temperature [C]']
#
## create dataset for tf, slice along first dimension: datapoint
## one element := one datapoint (RUN_XYZ); each datapoint is a dictionnary with [0]: input keys="physical properties" (p,pos_hp), values =(x,y,z)-component, [1]:  output keys="physical properties" (T), values =(x,y,z)-component
#tf_data = tf.data.Dataset.from_tensor_slices((data_in, data_labels))
##print(tf_data.element_spec)
#print([elem[0]['Liquid_Pressure [Pa]'][0,0,0].numpy() for elem in tf_data])
#
## shuffle the data
#tf_data = tf_data.shuffle(shuffle_buffer_size)
#print([elem[0]['Liquid_Pressure [Pa]'][0,0,0].numpy() for elem in tf_data])
#
##np.random.seed(seed=1) # TODO just for programming and varification purposes at this point, delete later!

In [300]:
## parameter setting
#train_size_percent = 0.75
#test_size_percent = 0.25
#
#np.random.seed(seed=1) # TODO just for programming and varification purposes at this point, delete later!
#
#def split_dataset(train_size_percent, test_size_percent, dataset_pressure, dataset_temperature, dataset_vel_x, dataset_vel_y, dataset_vel_z):
#    # calculating parameters from settings
#    dataset_size = len(dataset_pressure)
#    train_size = int(dataset_size * train_size_percent)
#    test_size = int(dataset_size * test_size_percent)
#
#    # create random split of data
#    indices = np.arange(0,100)
#    np.random.shuffle(indices)
#    indices_train = indices[:train_size]
#    indices_test = indices[train_size:train_size+test_size]
#
#    # split dataset into training and validation data 
#    data_train_pressure = []
#    data_train_temperature = []
#    data_train_vel_x = []
#    data_train_vel_y = []
#    data_train_vel_z = []
#
#    data_test_pressure = []
#    data_test_temperature = []
#    data_test_vel_x = []
#    data_test_vel_y = []
#    data_test_vel_z = []
#
#    for index in indices_train:
#        data_train_pressure.append(dataset_pressure[index])
#        data_train_temperature.append(dataset_temperature[index])
#        data_train_vel_x.append(dataset_vel_x[index])
#        data_train_vel_y.append(dataset_vel_y[index])
#        data_train_vel_z.append(dataset_vel_z[index])
#
#    for index in indices_test:
#        data_test_pressure.append(dataset_pressure[index])
#        data_test_temperature.append(dataset_temperature[index])
#        data_test_vel_x.append(dataset_vel_x[index])
#        data_test_vel_y.append(dataset_vel_y[index])
#        data_test_vel_z.append(dataset_vel_z[index])
#
#    data_train_pressure = np.array(data_train_pressure)
#    data_train_temperature = np.array(data_train_temperature)
#    data_train_vel_x = np.array(data_train_vel_x)
#    data_train_vel_y = np.array(data_train_vel_y)
#    data_train_vel_z = np.array(data_train_vel_z)
#    data_test_pressure = np.array(data_test_pressure)
#    data_test_temperature = np.array(data_test_temperature)
#    data_test_vel_x = np.array(data_test_vel_x)
#    data_test_vel_y = np.array(data_test_vel_y)
#    data_test_vel_z = np.array(data_test_vel_z)
#
#    return data_train_pressure, data_train_temperature, data_train_vel_x, data_train_vel_y, data_train_vel_z, data_test_pressure, data_test_temperature, data_test_vel_x, data_test_vel_y, data_test_vel_z

#data_train_pressure, data_train_temperature, data_train_vel_x, data_train_vel_y, data_train_vel_z, data_test_pressure, data_test_temperature, data_test_vel_x, data_test_vel_y, data_test_vel_z = split_dataset(train_size_percent, test_size_percent, dataset_pressure_init, dataset_temperature, dataset_vel_x, dataset_vel_y, dataset_vel_z)

**FCN model definition**

adapted from https://github.com/himanshurawlani/fully_convolutional_network.git
made it 3D

In [48]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Dropout, BatchNormalization, Activation
from tensorflow.keras.utils import to_categorical

In [68]:
kernel = 3#(3, 3, 3, 1)
def FCN_model(input_shape=(20,200,16,1), len_classes=5, dropout_rate=0.2, kernel=3):

    input = Input(shape=input_shape) #2D: (None, None, 3))
    x = Conv3D(filters=32, kernel_size=kernel, strides=1,padding="same")(input) 
    #  filters: Integer, the dimensionality of the output space (i.e. the number of output filters in the convolution).
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = Conv3D(filters=64, kernel_size=kernel, strides=1,padding= "same")(x)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = Conv3D(filters=128, kernel_size=kernel, strides=2)(x)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = Conv3D(filters=256, kernel_size=kernel, strides=2)(x)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    # x = Conv3D(filters=512, kernel_size=kernel, strides=2)(x)
    # x = Dropout(dropout_rate)(x)
    # x = BatchNormalization()(x)
    # x = Activation('relu')(x)

    # Uncomment the below line if you're using dense layers
    # x = tf.keras.layers.GlobalMaxPooling2D()(x)

    # Fully connected layer 1
    # x = tf.keras.layers.Dropout(dropout_rate)(x)
    # x = tf.keras.layers.BatchNormalization()(x)
    # x = tf.keras.layers.Dense(units=64)(x)
    # x = tf.keras.layers.Activation('relu')(x)

    # Fully connected layer 1
    x = Conv3D(filters=64, kernel_size=1, strides=1)(x)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Fully connected layer 2
    # x = tf.keras.layers.Dropout(dropout_rate)(x)
    # x = tf.keras.layers.BatchNormalization()(x)
    # x = tf.keras.layers.Dense(units=len_classes)(x)
    # predictions = tf.keras.layers.Activation('softmax')(x)

    # Fully connected layer 2
    x = Conv3D(filters=len_classes, kernel_size=1, strides=1)(x)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)
    #x = GlobalMaxPooling3D()(x) #TODO I PERSONALLY EXCLUDED - BACK IN?
    predictions = Activation('softmax')(x)

    model = tensorflow.keras.Model(inputs=input, outputs=predictions)
    
    print(model.summary())
    print(f'Total number of layers: {len(model.layers)}')

    return model


In [69]:
model = FCN_model(input_shape=(20, 200, 16, 1), len_classes=5, dropout_rate=0.2)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 20, 200, 16, 1)]  0         
                                                                 
 conv3d_37 (Conv3D)          (None, 20, 200, 16, 32)   896       
                                                                 
 dropout_28 (Dropout)        (None, 20, 200, 16, 32)   0         
                                                                 
 batch_normalization_28 (Bat  (None, 20, 200, 16, 32)  128       
 chNormalization)                                                
                                                                 
 activation_28 (Activation)  (None, 20, 200, 16, 32)   0         
                                                                 
 conv3d_38 (Conv3D)          (None, 20, 200, 16, 64)   55360     
                                                             